In [6]:
from pathlib import Path

import pandas as pd


def load_accident_files(years=[2021, 2022, 2023]):
    """
    Charge les données des accidents pour plusieurs années
    """
    all_years_data = {}

    file_patterns = {
        "caracteristiques": ["caract-{}.csv"],
        "lieux": ["lieux-{}.csv"],
        "vehicules": ["vehicules-{}.csv"],
        "usagers": ["usagers-{}.csv"],
    }

    for year in years:
        print(f"\nChargement des données de {year}")
        year_data = {}

        for table, patterns in file_patterns.items():
            for pattern in patterns:
                filename = pattern.format(year)
                if Path(filename).exists():
                    try:
                        df = pd.read_csv(filename, sep=";", low_memory=False, encoding="utf-8")
                        print(f"✓ Chargé {filename}: {df.shape[0]} lignes")

                        # Normaliser les noms de colonnes
                        df.columns = df.columns.str.lower().str.strip()

                        # Convertir num_acc en string pour les jointures
                        if "num_acc" in df.columns:
                            df["num_acc"] = df["num_acc"].astype(str)

                        if "num_veh" in df.columns:
                            df["num_veh"] = df["num_veh"].astype(str)

                        year_data[table] = df
                        break

                    except Exception as e:
                        print(f"Erreur avec {filename}: {str(e)}")
                        continue

        if year_data:
            all_years_data[year] = year_data

    return all_years_data


def merge_year_data(year_data, year):
    """
    Fusionne les tables pour une année donnée
    """
    try:
        # 1. Fusion caracteristiques et lieux
        merged = pd.merge(year_data["caracteristiques"], year_data["lieux"], on="num_acc", how="left")
        print(f"Étape 1 - Fusion caract-lieux: {merged.shape[0]} lignes")

        # 2. Fusion avec vehicules
        merged = pd.merge(merged, year_data["vehicules"], on="num_acc", how="left")
        print(f"Étape 2 - Fusion avec vehicules: {merged.shape[0]} lignes")

        # 3. Fusion avec usagers
        merged = pd.merge(merged, year_data["usagers"], on=["num_acc", "num_veh"], how="left")
        print(f"Étape 3 - Fusion avec usagers: {merged.shape[0]} lignes")

        # Ajouter l'année après la fusion
        merged["annee"] = year

        return merged

    except Exception as e:
        print(f"Erreur détaillée lors de la fusion: {str(e)}")
        # Afficher les colonnes disponibles pour le debug
        for name, df in year_data.items():
            print(f"\nColonnes dans {name}:")
            print(df.columns.tolist())
        return None


def combine_all_years(all_years_data):
    """
    Combine les données de toutes les années en un seul DataFrame
    """
    all_years_merged = []

    for year, year_data in all_years_data.items():
        print(f"\nTraitement de l'année {year}")
        year_merged = merge_year_data(year_data, year)
        if year_merged is not None:
            print(f"✓ Année {year} traitée: {year_merged.shape[0]} lignes")
            all_years_merged.append(year_merged)

    if all_years_merged:
        final_df = pd.concat(all_years_merged, ignore_index=True)
        print(f"\nJeu de données final: {final_df.shape[0]} lignes, {final_df.shape[1]} colonnes")
        return final_df
    else:
        print("Aucune donnée n'a pu être combinée")
        return None


def display_data_summary(df):
    """
    Affiche un résumé des données combinées
    """
    print("\nRésumé des données:")
    print(f"Période couverte: {df['annee'].min()} - {df['annee'].max()}")
    print("\nNombre d'accidents par année:")
    print(df["annee"].value_counts().sort_index())

    if "grav" in df.columns:
        print("\nDistribution de la gravité des accidents:")
        print(df["grav"].value_counts())

    print("\nColonnes disponibles:", len(df.columns))
    print("\nÉchantillon des colonnes:", ", ".join(sorted(df.columns)[:10]))

In [7]:
# Charger les données
all_years_data = load_accident_files([2021, 2022, 2023])

# Combiner les années
combined_data = combine_all_years(all_years_data)

# Afficher le résumé si la combinaison a réussi
if combined_data is not None:
    display_data_summary(combined_data)


Chargement des données de 2021
✓ Chargé caract-2021.csv: 56518 lignes
✓ Chargé lieux-2021.csv: 56518 lignes
✓ Chargé vehicules-2021.csv: 97315 lignes
✓ Chargé usagers-2021.csv: 129248 lignes

Chargement des données de 2022
✓ Chargé caract-2022.csv: 55302 lignes
✓ Chargé lieux-2022.csv: 55302 lignes
✓ Chargé vehicules-2022.csv: 94493 lignes
✓ Chargé usagers-2022.csv: 126662 lignes

Chargement des données de 2023
✓ Chargé caract-2023.csv: 54822 lignes
✓ Chargé lieux-2023.csv: 70860 lignes
✓ Chargé vehicules-2023.csv: 93585 lignes
✓ Chargé usagers-2023.csv: 125789 lignes

Traitement de l'année 2021
Étape 1 - Fusion caract-lieux: 56518 lignes
Étape 2 - Fusion avec vehicules: 97315 lignes
Étape 3 - Fusion avec usagers: 129254 lignes
✓ Année 2021 traitée: 129254 lignes

Traitement de l'année 2022
Erreur détaillée lors de la fusion: 'num_acc'

Colonnes dans caracteristiques:
['accident_id', 'jour', 'mois', 'an', 'hrmn', 'lum', 'dep', 'com', 'agg', 'int', 'atm', 'col', 'adr', 'lat', 'long']



In [9]:
combined_data.to_csv("data_combined.csv", sep=";", index=False, encoding="utf-8")

In [10]:
combined_data.head()

,num_acc,jour,mois,an,hrmn,lum,dep,com,agg,int,...,sexe,an_nais,trajet,secu1,secu2,secu3,locp,actp,etatp,annee
0,202100000001,30,11,2021,07:32,2,30,30319,1,1,...,1.0,2000.0,1.0,0.0,9.0,-1.0,0.0,0,-1.0,2021
1,202100000001,30,11,2021,07:32,2,30,30319,1,1,...,1.0,1978.0,1.0,1.0,-1.0,-1.0,0.0,0,-1.0,2021
2,202100000002,25,9,2021,14:20,1,51,51544,1,3,...,1.0,1983.0,0.0,1.0,-1.0,-1.0,0.0,0,-1.0,2021
3,202100000002,25,9,2021,14:20,1,51,51544,1,3,...,1.0,1993.0,0.0,1.0,-1.0,-1.0,0.0,0,-1.0,2021
4,202100000003,15,7,2021,07:55,1,85,85048,2,1,...,1.0,1995.0,1.0,1.0,0.0,-1.0,0.0,0,-1.0,2021


In [13]:
combined_data.shape

(292988, 57)